In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector as sq
con=sq.connect(host='localhost',user='root',passwd='Zodiac@3006',database='retail_inventory',allow_local_infile=True,autocommit=True )
if con.is_connected():
        print(  'success')
cur=con.cursor()
cur.execute("CREATE DATABASE IF NOT EXISTS retail_inventory")
cur.execute("USE retail_inventory")
csv_path = r"C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\inventory_forecasting.csv"
cur.execute("SET GLOBAL local_infile = 1")
# cur.execute("SET SESSION local_infile = 1")
cur.execute("""
    CREATE TABLE IF NOT EXISTS inventory_raw (
        Date DATE,
        Store_ID VARCHAR(10),
        Product_ID VARCHAR(10),
        Category VARCHAR(50),
        Region VARCHAR(50),
        Inventory_Level INT,
        Units_Sold INT,
        Units_Ordered INT,
        Demand_Forecast DECIMAL(10,2),
        Price DECIMAL(10,2),
        Discount INT,
        Weather_Condition VARCHAR(50),
        Holiday_Promotion BOOLEAN,
        Competitor_Pricing DECIMAL(10,2),
        Seasonality VARCHAR(20)
    )
""")
print(" Table `inventory_raw` created")

# Step 4: Load CSV into the table
load_query = f"""
    LOAD DATA LOCAL INFILE '{csv_path}'
    INTO TABLE inventory_raw
    FIELDS TERMINATED BY ',' 
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 ROWS;
"""

try:
    cur.execute(load_query)
    con.commit()
    print(" Data loaded successfully into inventory_raw")
except Exception as e:
    print(" Failed to load data:", e)
cur.execute("SELECT * FROM inventory_raw LIMIT 5")
for row in cur.fetchall():
    print(row)

con.commit()
cur.close()
con.close()

success
 Table `inventory_raw` created
 Data loaded successfully into inventory_raw
(datetime.date(2022, 1, 1), 'S001', 'P0096', 'Toys', 'West', 158, 134, 142, Decimal('152.36'), Decimal('40.88'), 5, 'Sunny', 1, Decimal('42.39'), 'Winter')
(datetime.date(2022, 1, 1), 'S001', 'P0016', 'Clothing', 'East', 189, 127, 125, Decimal('150.47'), Decimal('90.78'), 0, 'Rainy', 0, Decimal('84.16'), 'Winter')
(datetime.date(2022, 1, 1), 'S001', 'P0031', 'Electronics', 'West', 75, 48, 39, Decimal('68.62'), Decimal('13.99'), 20, 'Rainy', 0, Decimal('12.67'), 'Winter')
(datetime.date(2022, 1, 1), 'S001', 'P0159', 'Electronics', 'West', 161, 74, 71, Decimal('75.46'), Decimal('84.92'), 10, 'Snowy', 0, Decimal('83.03'), 'Winter')
(datetime.date(2022, 1, 1), 'S001', 'P0129', 'Furniture', 'South', 135, 66, 69, Decimal('67.18'), Decimal('19.66'), 0, 'Cloudy', 0, Decimal('20.55'), 'Winter')


Relational Schema

In [3]:
import mysql.connector as sq
con=sq.connect(host='localhost',user='root',passwd='Zodiac@3006',database='retail_inventory',allow_local_infile=True,autocommit=True )
cur=con.cursor()
# Step 2: Drop old tables if re-running
tables = ["Forecasting", "External_Factors", "Pricing", "Inventory", "Products", "Stores"]
for t in tables:
    cur.execute(f"DROP TABLE IF EXISTS {t}")

# Step 3: Final schema with primary keys, FKs, unique constraints
schema_queries = [
    """
    CREATE TABLE Stores (
        Store_ID VARCHAR(10) PRIMARY KEY,
        Region VARCHAR(50)
    )
    """,

    """
    CREATE TABLE Products (
        Product_ID VARCHAR(10) PRIMARY KEY,
        Category VARCHAR(50)
    )
    """,

    """
    CREATE TABLE Inventory (
        Inventory_ID INT AUTO_INCREMENT PRIMARY KEY,
        Date DATE NOT NULL,
        Store_ID VARCHAR(10),
        Product_ID VARCHAR(10),
        Inventory_Level INT,
        Units_Sold INT,
        Units_Ordered INT,
        FOREIGN KEY (Store_ID) REFERENCES Stores(Store_ID),
        FOREIGN KEY (Product_ID) REFERENCES Products(Product_ID),
        UNIQUE (Date, Store_ID, Product_ID)
    )
    """,

    """
    CREATE TABLE Pricing (
        Pricing_ID INT AUTO_INCREMENT PRIMARY KEY,
        Date DATE NOT NULL,
        Product_ID VARCHAR(10),
        Price DECIMAL(10,2),
        Discount INT,
        Competitor_Pricing DECIMAL(10,2),
        FOREIGN KEY (Product_ID) REFERENCES Products(Product_ID),
        UNIQUE (Date, Product_ID)
    )
    """,

    """
    CREATE TABLE External_Factors (
        Factor_ID INT AUTO_INCREMENT PRIMARY KEY,
        Date DATE NOT NULL,
        Store_ID VARCHAR(10),
        Weather_Condition VARCHAR(50),
        Holiday_Promotion BOOLEAN,
        Seasonality VARCHAR(20),
        FOREIGN KEY (Store_ID) REFERENCES Stores(Store_ID),
        UNIQUE (Date, Store_ID)
    )
    """,

    """
    CREATE TABLE Forecasting (
        Date DATE NOT NULL,
        Store_ID VARCHAR(10),
        Product_ID VARCHAR(10),
        Demand_Forecast DECIMAL(10,2),
        PRIMARY KEY (Date, Store_ID, Product_ID),
        FOREIGN KEY (Store_ID) REFERENCES Stores(Store_ID),
        FOREIGN KEY (Product_ID) REFERENCES Products(Product_ID)
    )
    """
]

# Step 4: Execute all table creation queries
for query in schema_queries:
    cur.execute(query)

con.commit()
print("Final schema with primary keys and foreign keys created successfully.")
# con.commit()
cur.close()
con.close()

Final schema with primary keys and foreign keys created successfully.


In [4]:
import mysql.connector as sq
con=sq.connect(host='localhost',user='root',passwd='Zodiac@3006',database='retail_inventory',allow_local_infile=True,autocommit=True )
cur=con.cursor()
cur.execute("""
INSERT IGNORE INTO Stores (Store_ID, Region)
SELECT DISTINCT Store_ID, Region FROM inventory_raw;
""")

# 2. Populate Products
cur.execute("""
INSERT IGNORE INTO Products (Product_ID, Category)
SELECT DISTINCT Product_ID, Category FROM inventory_raw;
""")

# 3. Populate Inventory
cur.execute("""
INSERT INTO Inventory (Date, Store_ID, Product_ID, Inventory_Level, Units_Sold, Units_Ordered)
SELECT Date, Store_ID, Product_ID, Inventory_Level, Units_Sold, Units_Ordered
FROM inventory_raw
ON DUPLICATE KEY UPDATE
    Inventory_Level = VALUES(Inventory_Level),
    Units_Sold = VALUES(Units_Sold),
    Units_Ordered = VALUES(Units_Ordered);
""")

# 4. Populate Pricing
cur.execute("""
INSERT INTO Pricing (Date, Product_ID, Price, Discount, Competitor_Pricing)
SELECT Date, Product_ID, Price, Discount, Competitor_Pricing
FROM inventory_raw
ON DUPLICATE KEY UPDATE
    Price = VALUES(Price),
    Discount = VALUES(Discount),
    Competitor_Pricing = VALUES(Competitor_Pricing);
""")

# 5. Populate External_Factors
cur.execute("""
INSERT INTO External_Factors (Date, Store_ID, Weather_Condition, Holiday_Promotion, Seasonality)
SELECT Date, Store_ID, Weather_Condition, Holiday_Promotion, Seasonality
FROM inventory_raw
ON DUPLICATE KEY UPDATE
    Weather_Condition = VALUES(Weather_Condition),
    Holiday_Promotion = VALUES(Holiday_Promotion),
    Seasonality = VALUES(Seasonality);
""")

# 6. Populate Forecasting
cur.execute("""
INSERT INTO Forecasting (Date, Store_ID, Product_ID, Demand_Forecast)
SELECT Date, Store_ID, Product_ID, Demand_Forecast
FROM inventory_raw
ON DUPLICATE KEY UPDATE
    Demand_Forecast = VALUES(Demand_Forecast);
""")

con.commit()
print(" All normalized tables successfully populated from inventory_raw.")

cur.close()
con.close()

 All normalized tables successfully populated from inventory_raw.


SQL Queries

In [5]:
import mysql.connector as sq
import pandas as pd

# Connect to your MySQL server
con = sq.connect(
    host='localhost',
    user='root',
    passwd='Zodiac@3006',
    database='retail_inventory',
    autocommit=True
)
cur = con.cursor()

# Define analysis queries
queries = {
    "Stock Level Summary(Stores) ": """
       SELECT
    Date,
    Product_ID,
    SUM(Inventory_Level) AS Total_Stock_Per_Product_On_Date
FROM
    inventory_raw
GROUP BY
    Date, Product_ID
ORDER BY
    Date ASC, Product_ID ASC;
    """,
    "Stock Level Summary (Regions)":"""
SELECT
    Date,
    Region,
    Product_ID,
    SUM(Inventory_Level) AS Total_Stock_Per_Product_On_Date_In_Region
FROM
    inventory_raw
GROUP BY
    Date, Region, Product_ID
ORDER BY
    Date ASC, Region ASC, Product_ID ASC;    
""",

    "Low Inventory Detection (below 50 units)": """
        SELECT 
    Date,
    Store_ID,
    Product_ID,
    Inventory_Level
FROM Inventory
WHERE Inventory_Level < 50
ORDER BY Inventory_Level ASC;
    """,

   "Reorder Point":"""
WITH DailySales AS (
    SELECT
        i.Date,
        i.Store_ID,
        i.Product_ID,
        ef.Seasonality,
        SUM(i.Units_Sold) AS Daily_Units_Sold
    FROM Inventory i
    JOIN External_Factors ef
        ON i.Date = ef.Date AND i.Store_ID = ef.Store_ID
    GROUP BY i.Date, i.Store_ID, i.Product_ID, ef.Seasonality
),

SeasonalAvgDemand AS (
    SELECT
        Store_ID,
        Product_ID,
        Seasonality,
        AVG(Daily_Units_Sold) AS Avg_Daily_Demand
    FROM DailySales
    WHERE Date BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY Store_ID, Product_ID, Seasonality
)

SELECT
    i.Date,
    i.Store_ID,
    i.Product_ID,
    p.Category,
    ef.Seasonality,
    i.Inventory_Level,
    i.Units_Sold,

    sad.Avg_Daily_Demand,

    CEIL(sad.Avg_Daily_Demand * 1.2) AS Reorder_Point,

    CASE
        WHEN i.Inventory_Level < CEIL(sad.Avg_Daily_Demand * 1.2)
            THEN 'LOW STOCK - Reorder Recommended'
        ELSE 'Sufficient Stock'
    END AS Inventory_Status,

    CASE
        WHEN i.Inventory_Level < CEIL(sad.Avg_Daily_Demand * 1.2)
            THEN ROUND(((CEIL(sad.Avg_Daily_Demand * 1.2) - i.Inventory_Level) / 
                        CEIL(sad.Avg_Daily_Demand * 1.2)) * 100, 2)
        ELSE 0
    END AS Shortfall_Percentage

FROM Inventory i
JOIN External_Factors ef
    ON i.Date = ef.Date AND i.Store_ID = ef.Store_ID
JOIN Products p
    ON i.Product_ID = p.Product_ID
JOIN SeasonalAvgDemand sad
    ON i.Product_ID = sad.Product_ID
    AND i.Store_ID = sad.Store_ID
    AND ef.Seasonality = sad.Seasonality

ORDER BY i.Date DESC, Shortfall_Percentage DESC;
""",

    "Fast and Slow moving Products":'''

WITH InventoryYearly AS (
    SELECT
        Product_ID,
        EXTRACT(YEAR FROM Date) AS Year,
        AVG(Inventory_Level) AS Avg_Inventory
    FROM inventory
    WHERE EXTRACT(YEAR FROM Date) IN (2022, 2023)
    GROUP BY Product_ID, EXTRACT(YEAR FROM Date)
),
SalesYearly AS (
    SELECT
        Product_ID,
        EXTRACT(YEAR FROM Date) AS Year,
        SUM(Units_Sold) AS Total_Units_Sold
    FROM inventory
    WHERE EXTRACT(YEAR FROM Date) IN (2022, 2023)
    GROUP BY Product_ID, EXTRACT(YEAR FROM Date)
),
PriceYearly AS (
    SELECT
        Product_ID,
        EXTRACT(YEAR FROM Date) AS Year,
        AVG(Price) AS Avg_Unit_Price
    FROM pricing
    WHERE EXTRACT(YEAR FROM Date) IN (2022, 2023)
    GROUP BY Product_ID, EXTRACT(YEAR FROM Date)
),
TurnoverCalc AS (
    SELECT
        s.Product_ID,
        s.Year,
        s.Total_Units_Sold,
        iy.Avg_Inventory,
        py.Avg_Unit_Price,
        ROUND(s.Total_Units_Sold * py.Avg_Unit_Price, 2) AS Revenue,
        ROUND(iy.Avg_Inventory * py.Avg_Unit_Price, 2) AS Avg_Inventory_Value,
        CASE 
            WHEN (iy.Avg_Inventory * py.Avg_Unit_Price) > 0 THEN
                ROUND((s.Total_Units_Sold * py.Avg_Unit_Price) / (iy.Avg_Inventory * py.Avg_Unit_Price), 2)
            ELSE NULL
        END AS Inventory_Turnover_Ratio
    FROM SalesYearly s
    JOIN InventoryYearly iy ON s.Product_ID = iy.Product_ID AND s.Year = iy.Year
    JOIN PriceYearly py ON s.Product_ID = py.Product_ID AND s.Year = py.Year
),
AverageTurnover AS (
    SELECT
        ROUND(AVG(Inventory_Turnover_Ratio), 2) AS Avg_Turnover
    FROM TurnoverCalc
    WHERE Inventory_Turnover_Ratio IS NOT NULL
)

SELECT
    t.Year,
    t.Product_ID,
    t.Total_Units_Sold,
    t.Revenue,
    t.Avg_Inventory_Value,
    t.Inventory_Turnover_Ratio,
    a.Avg_Turnover,
    CASE
        WHEN t.Inventory_Turnover_Ratio IS NULL THEN 'No Data'
        WHEN t.Inventory_Turnover_Ratio >= a.Avg_Turnover  THEN 'Fast-Moving (Above Avg)'
        WHEN t.Inventory_Turnover_Ratio <= a.Avg_Turnover THEN 'Slow-Moving (Below Avg)'
        ELSE 'Moderate-Moving (±15% of Avg)'
    END AS Movement_Category
FROM TurnoverCalc t
CROSS JOIN AverageTurnover a
ORDER BY t.Year, t.Inventory_Turnover_Ratio DESC;''',

}

# Execute and print results
for title, query in queries.items():
    cur.execute(query)
    data = cur.fetchall()
    df = pd.DataFrame(data, columns=[desc[0] for desc in cur.description])
    print(f"\n {title}")
    print(df)

cur.close()
con.close()



 Stock Level Summary(Stores) 
             Date Product_ID Total_Stock_Per_Product_On_Date
0      2022-01-01      P0016                           18278
1      2022-01-01      P0017                           11932
2      2022-01-01      P0031                           12635
3      2022-01-01      P0046                           16397
4      2022-01-01      P0057                           17879
...           ...        ...                             ...
21895  2023-12-31      P0171                           15751
21896  2023-12-31      P0175                           11685
21897  2023-12-31      P0178                           16207
21898  2023-12-31      P0183                           15808
21899  2023-12-31      P0187                           16682

[21900 rows x 3 columns]

 Stock Level Summary (Regions)
             Date Region Product_ID Total_Stock_Per_Product_On_Date_In_Region
0      2022-01-01   East      P0016                                     11552
1      2022-01-01   Eas

In [6]:
import mysql.connector as sq
import pandas as pd

# Connect to your MySQL database
con = sq.connect(
    host='localhost',
    user='root',
    passwd='Zodiac@3006',
    database='retail_inventory',
    autocommit=True
)
cur = con.cursor()

# Define queries
forecast_queries = {
    "Forecast Accuracy (MAPE) by Product": """
        SELECT 
            F.Product_ID,
            ROUND(AVG(ABS(F.Demand_Forecast - I.Units_Sold) / NULLIF(I.Units_Sold, 0)) * 100, 2) AS MAPE
        FROM Forecasting F
        JOIN Inventory I ON F.Date = I.Date AND F.Store_ID = I.Store_ID AND F.Product_ID = I.Product_ID
        GROUP BY F.Product_ID
        HAVING COUNT(*) > 5
        ORDER BY MAPE ASC
        LIMIT 10;
    """,

    "Over-Forecasting Risk (Forecast >> Sales)": """
        SELECT 
            F.Product_ID,
            COUNT(*) AS Instances,
            ROUND(AVG(F.Demand_Forecast - I.Units_Sold), 2) AS Avg_OverForecast
        FROM Forecasting F
        JOIN Inventory I ON F.Date = I.Date AND F.Store_ID = I.Store_ID AND F.Product_ID = I.Product_ID
        WHERE F.Demand_Forecast > I.Units_Sold * 1.3
        GROUP BY F.Product_ID
        ORDER BY Avg_OverForecast DESC
        LIMIT 10;
    """,

    "Under-Forecasting Risk (Sales >> Forecast)": """
        SELECT 
            F.Product_ID,
            COUNT(*) AS Instances,
            ROUND(AVG(I.Units_Sold - F.Demand_Forecast), 2) AS Avg_UnderForecast
        FROM Forecasting F
        JOIN Inventory I ON F.Date = I.Date AND F.Store_ID = I.Store_ID AND F.Product_ID = I.Product_ID
        WHERE I.Units_Sold > F.Demand_Forecast * 1.3
        GROUP BY F.Product_ID
        ORDER BY Avg_UnderForecast DESC
        LIMIT 10;
    """
}

# Execute and print results
for title, query in forecast_queries.items():
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    print(f"\n📈 {title}")
    print(df)

cur.close()
con.close()



📈 Forecast Accuracy (MAPE) by Product
  Product_ID   MAPE
0      P0067  15.44
1      P0166  15.46
2      P0153  15.53
3      P0066  15.61
4      P0096  15.64
5      P0178  15.69
6      P0116  15.71
7      P0129  15.71
8      P0175  15.71
9      P0057  15.74

📈 Over-Forecasting Risk (Forecast >> Sales)
  Product_ID  Instances Avg_OverForecast
0      P0057        684            31.34
1      P0187        683            31.21
2      P0066        645            31.12
3      P0046        702            31.08
4      P0133        650            31.01
5      P0125        673            30.89
6      P0016        667            30.80
7      P0069        674            30.66
8      P0178        667            30.51
9      P0126        670            30.49

📈 Under-Forecasting Risk (Sales >> Forecast)
Empty DataFrame
Columns: [Product_ID, Instances, Avg_UnderForecast]
Index: []
